In [1]:
# Install dependencies
!pip install langchain langchain-community faiss-cpu pypdf sentence-transformers transformers accelerate gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [4]:
# Imports
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
import gradio as gr



In [5]:
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving Medical_Book.pdf to Medical_Book.pdf


In [6]:
# Load PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split text into chunks (smaller to avoid >512 token input)
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vector DB
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()


/tmp/ipython-input-660727267.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

In [9]:
# Load Hugging Face model locally (no API calls)
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=512,
    device=0  # CPU (set 0 if GPU is available)
)

# Wrap pipeline into LangChain
llm = HuggingFacePipeline(pipeline=generator)



Device set to use cpu


In [10]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False   # ✅ only one output now
)

def ask_bot(query):
    try:
        result = qa.run(query)  # works fine now
        return result + "\n\n⚠️ Disclaimer: This is AI from Gale Encyclopedia, not medical advice."
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


In [11]:
iface = gr.Interface(
    fn=ask_bot,
    inputs="text",
    outputs="text",
    title="🩺 Medical Chatbot (PDF-based)",
    description="Ask me medical questions based on the Gale Encyclopedia of Medicine PDF. ⚠️ Not a substitute for professional medical advice."
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://31f0ea34a9fbb5f1b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
